## Обработка естественного языка (natural language processing)

In [1]:
import pandas as pd
df = pd.read_csv('IMDB Dataset.csv')
df.head(3)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive


In [2]:
df.shape

(50000, 2)

Перемешать набор данных:

In [3]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.head()

,review,sentiment
11841,John Cassavetes is on the run from the law. He...,positive
19602,It's not just that the movie is lame. It's mor...,negative
45519,"Well, if it weren't for Ethel Waters and a 7-y...",negative
25747,I find Alan Jacobs review very accurate concer...,positive
42642,This movie is simply awesome. It is so hilario...,positive


#### Трансформирование слов в векторы признаков (на основе счетчиков слов в соответствующих документах):

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array(['The sun is shining',
                 'The weather is sweet',
                 'The sun is shining, the weather is sweet, '
                 'and one and one is two'])
bag =  count.fit_transform(docs)

In [5]:
count.vocabulary_

{'the': 6,
 'sun': 4,
 'is': 1,
 'shining': 3,
 'weather': 8,
 'sweet': 5,
 'and': 0,
 'one': 2,
 'two': 7}

In [6]:
count.get_feature_names_out()

array(['and', 'is', 'one', 'shining', 'sun', 'sweet', 'the', 'two',
       'weather'], dtype=object)

In [7]:
bag.toarray()

array([[0, 1, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 0, 1],
       [2, 3, 2, 1, 1, 1, 2, 1, 1]], dtype=int64)

Значения в векторах матрицы признаков называются сырыми частотами термов (raw term frequency) tf(t, d) - сколько раз терм t встречается в документе d. Порядок слов или термов в предложении и документе не играет роли. Порядок, в котором частоты термов расположены в векторе признаков, определяются индексами в словаре, обычно по алфавиту.

Последовательность элементов является 1-граммой. Соприкасающаяся последовательность элементов в NLP - слов, букв или символов - называется n-граммой.

In [8]:
docs[0]

'The sun is shining'

In [9]:
count2 = CountVectorizer(ngram_range=(2, 2))
bag2 = count2.fit_transform([docs[0]])
count2.get_feature_names_out()

array(['is shining', 'sun is', 'the sun'], dtype=object)

#### Оценка важности слов с помощью приема tf-idf

Частота терма - обратная частота документа.
Данная мера может применяться для снижения веса часто встречающихся слов, которые не содержат полезной информации, в векторах признаков.

Мера tf-idf может быть определена, как произведение частоты терма на обратную частоту документа:

<img src="pic/tfidf.png" width="190"/>
tf(t,d) - частота терма, idf(t,d) - обратная частота документа:
<img src="pic/idf.png" width="190"/>

1. $n_d$ - общееколичество документов
2. частота документа $df(d,t)$ - количество документов d, содержащих терм t.

3. 1 в знаменателе служит для того, чтобы присвоить ненулевое значение термам, которые встречаются во всех обучающих образцах.

4. log используется для того, чтобы гарантировать, что низким частотам документов не назначится слишком большой вес.

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True,
                         norm='l2',
                         smooth_idf=True)  # idf(t,d) + 1
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


smooth_idf=True - полезна для назначения нулевых весов термам, встречающимся во всех документах.

Привычно проводить нормализацию сырых частот термов перед вычислением мер tf-idf:
$$v_{norm} = \frac{v}{||v||_2} = \frac{v}{\sqrt{v_1^2+v_2^2+...+v_n^2}}$$

#### Очистка текстовых данных

Необходимо удалить HTML-разметку, а также знаки препинания (кроме эмотиконов вроде смайликов) и другие небуквенные символы.

In [11]:
df.loc[1, 'review']

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [12]:
import re

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(<?::|;|=) (?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower() +
                   ' '.join(emoticons).replace('-', '')))
    return text
    
print(preprocessor(df.loc[1, 'review']))

a wonderful little production the filming technique is very unassuming very old time bbc fashion and gives a comforting and sometimes discomforting sense of realism to the entire piece the actors are extremely well chosen michael sheen not only has got all the polari but he has all the voices down pat too you can truly see the seamless editing guided by the references to williams diary entries not only is it well worth the watching but it is a terrificly written and performed piece a masterful production about one of the great master s of comedy and his life the realism really comes home with the little things the fantasy of the guard which rather than use the traditional dream techniques remains solid then disappears it plays on our knowledge and our senses particularly with the scenes concerning orton and halliwell and the sets particularly of their flat with halliwell s murals decorating every surface are terribly well done 


Применение функции preprocessor ко всем данным:

In [13]:
df['review'] = df['review'].apply(preprocessor)
df.head()

,review,sentiment
11841,john cassavetes is on the run from the law he ...,positive
19602,it s not just that the movie is lame it s more...,negative
45519,well if it weren t for ethel waters and a 7 ye...,negative
25747,i find alan jacobs review very accurate concer...,positive
42642,this movie is simply awesome it is so hilariou...,positive


#### Переработка документов в лексемы

Необходимо определить каким образом разделить совокупность текстов на индивидуальные элементы. Самый элементарный вариант разделение очищенных документов на отдельные слова по пробельным символам:

In [14]:
def tokenizer(text):
    return text.split()

print(tokenizer('runners like running and thus they run'))

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']


Word stemming - стэмминг слов.

Нахождение основы слова, это процесс, который представляет собой трансформирование словав в его корневую форму.

In [15]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

print(tokenizer_porter('runners like running and thus they run'))

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']


Удаление стоп слов (таких, как: is, and, has). Удаление стоп-слов может пригодиться, когда приходится иметь дело с сырыми или нормализованными частотами термов вместо мер tf-idf, в которых веса часто встречающихся слов уже понижены.

Для удаления стоп-слов будет использоваться набор из 127 стоп-слов английского языка:

In [16]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zekat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot') if
 w not in stop]

['runner', 'like', 'run', 'run', 'lot']

#### Обучение логистической регрессионной модели для классификации документов

Разбитие данных на обучающий и тестовый наборы:

In [18]:
X_train = df.head(25000).loc[:, 'review'].values
y_train = df.head(25000).loc[:, 'sentiment'].values

X_test = df.tail(25000).loc[:, 'review'].values
y_test = df.tail(25000).loc[:, 'sentiment'].values

In [19]:
from collections import Counter

print('train: ', Counter(y_train))
print('test: ', Counter(y_test))

train:  Counter({'negative': 12561, 'positive': 12439})
test:  Counter({'positive': 12561, 'negative': 12439})


In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
# TfidfVectorizer объединяет CountVectorizer и TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)
lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0,
                                                solver='liblinear'))])

lr_tfidf.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__norm', 'vect__preprocessor', 'vect__smooth_idf', 'vect__stop_words', 'vect__strip_accents', 'vect__sublinear_tf', 'vect__token_pattern', 'vect__tokenizer', 'vect__use_idf', 'vect__vocabulary', 'clf__C', 'clf__class_weight', 'clf__dual', 'clf__fit_intercept', 'clf__intercept_scaling', 'clf__l1_ratio', 'clf__max_iter', 'clf__multi_class', 'clf__n_jobs', 'clf__penalty', 'clf__random_state', 'clf__solver', 'clf__tol', 'clf__verbose', 'clf__warm_start'])

In [21]:
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer],
               'vect__use_idf': [False],
               'vect__norm': [None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]}]

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5, verbose=3,
                           n_jobs=-1)
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0,
                                                           solver='liblinear'))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've...
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you'

In [22]:
# [tokenizer, tokenizer_porter]

In [23]:
print(gs_lr_tfidf.best_params_)

{'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x0000022C25E5D9D0>}


In [24]:
print(gs_lr_tfidf.best_score_)

0.89672


In [25]:
clf = gs_lr_tfidf.best_estimator_
print(clf.score(X_test, y_test))

0.89952
